In [38]:
import numpy as np
import pandas as pd
#import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 

In [39]:
def dayOfYear(month, day, year):
    daysInMonth = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
    dayOfYear = sum(daysInMonth[:int(month)-1]) + int(day)
    if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) and dayOfYear > 59:
        dayOfYear -= 1
    return dayOfYear

def preprocessing(df):
    
    df['ArrivalDayOfYear'] = df.apply(lambda row: dayOfYear(row['ArrivalMonth'], row['ArrivalDate'], row['ArrivalYear']), axis=1)
    
    df['TotalNights'] = df['NumWeekendNights'] + df['NumWeekNights']

    df['TotalGuests'] = df['NumAdults'] + df['NumChildren']
    
    df['TotalPrice'] = df['AvgRoomPrice'] * df['TotalNights']



In [40]:
df = pd.read_csv('train_data.csv')
preprocessing(df)
categories = ['MealPlan', 'RoomType', 'MarketSegment', 'BookingStatus']
dummy_vars = pd.get_dummies(df[categories])
df = pd.concat([df, dummy_vars], axis=1)
df = df.drop(['MealPlan', 'RoomType', 'MarketSegment', 'BookingStatus', 'BookingStatus_Canceled'], axis=1)

print(df)

       BookingID  LeadTime  ArrivalYear  ArrivalMonth  ArrivalDate  \
0              1        10         2018             3           31   
1              2       116         2018             2           28   
2              3        11         2018             7           25   
3              4         3         2017             9           12   
4              5        28         2018             3            7   
...          ...       ...          ...           ...          ...   
29015      29016         2         2018            11           21   
29016      29017        18         2018             4           12   
29017      29018         8         2017             9           29   
29018      29019        20         2018             9           18   
29019      29020       274         2018             7           31   

       NumWeekendNights  NumWeekNights  Parking  NumAdults  NumChildren  ...  \
0                     0              1        0          1            0  ...   

In [41]:
#import seaborn as sns
#import matplotlib.pyplot as plt

#plt.subplots(figsize=(50, 35))
#sns.heatmap(df.corr(), annot=True)


In [42]:
X = df.drop(['BookingStatus_Not_Canceled', 
             'BookingID'], axis=1) # features
y = df['BookingStatus_Not_Canceled'] # target value

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [44]:
X_train.values

array([[4.000e+00, 2.018e+03, 4.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [6.000e+01, 2.018e+03, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.020e+02, 2.017e+03, 1.000e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.300e+02, 2.018e+03, 9.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [1.910e+02, 2.018e+03, 6.000e+00, ..., 0.000e+00, 0.000e+00,
        1.000e+00],
       [9.700e+01, 2.018e+03, 1.000e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00]])

In [45]:
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

pipelines = {
    'logistic regression:':make_pipeline(StandardScaler(), LogisticRegression()),
    'ridge classifier:':make_pipeline(StandardScaler(), RidgeClassifier()),
    'random forest classifier:':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gradient boosting classifier:':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
    'SVM:':make_pipeline(StandardScaler(), SVC())
}

In [46]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train.values, y_train)
    fit_models[algo] = model

C:\Users\xande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [47]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

C:\Users\xande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\xande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\xande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
C:\Users\xande\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\base.py:432: UserWarning: X has feature names, but StandardScale

logistic regression: 0.8087525844245348
ridge classifier: 0.8023776705720193
random forest classifier: 0.8976567884217781
gradient boosting classifier: 0.8544107512060648
SVM: 0.8368366643694004
